In [1]:
import yfinance as yf
import pandas as pd

In [2]:
stock_data = yf.download("DKNG", start="2020-01-01", end="2024-01-01")

# Save relevant features: Close price, Volume, Daily Returns
stock_data['Daily Return'] = stock_data['Close'].pct_change()
stock_data.dropna(inplace=True)

# Save the data for later use
stock_data.to_csv("draftkings_data.csv")

print(stock_data.head())

[*********************100%***********************]  1 of 1 completed

Price       Close   High     Low   Open   Volume Daily Return
Ticker       DKNG   DKNG    DKNG   DKNG     DKNG             
Date                                                         
2020-01-03  10.68  10.75  10.610  10.66   467600     0.000000
2020-01-06  10.80  10.90  10.692  10.75  2183500     0.011236
2020-01-07  10.90  10.95  10.870  10.89  1977000     0.009259
2020-01-08  11.26  11.35  10.884  10.94  2955700     0.033028
2020-01-09  12.25  12.39  11.340  11.43  8279300     0.087922
